# Local Evaluation - Groundedness

After you have setup and configured the prompt flow, its time to evaluation its performance. Here we can use the prompt flow SDK to test different questions and see how the prompt flow performs using the evaluation prompt flows provided.

In [2]:
from promptflow import PFClient

pf_client = PFClient()

from dotenv import load_dotenv

load_dotenv()

In [3]:
# Add a question to test the base prompt flow.
question = "Can you tell me about your jackets?"
customerId = "4"
output = pf_client.test(
    flow="../contoso-chat", # Path to the flow directory
    inputs={ # Inputs to the flow
        "chat_history": [],
        "question": question,
        "customerId": customerId,
    },
)

output["answer"] = "".join(list(output["answer"]))

In [4]:
output

Test the groundedness of the prompt flow with the answer from the above question.

In [5]:
test = pf_client.test(
    flow="groundedness",
    inputs={
        "question": question,
        "context": str(output["context"]),
        "answer": output["answer"],
    },
)

In [6]:
test

# Local Evaluation - Multiple Metrics 

Now use the same prompt flow and test it against the Multi Evaluation flow for groundedness, coherence, fluency, and relevance.

In [7]:
test_multi = pf_client.test(
    "multi_flow",
    inputs={
        "question": question,
        "context": str(output["context"]),
        "answer": output["answer"],
    },
)

In [8]:
test_multi

# AI Studio Azure batch run on an evaluation json dataset

Now in order to test these more thoroughly, we can use the Azure AI Studio to run batches of test data with the evaluation prompt flow on a larger dataset.

In [9]:
import json
# Import required libraries
from promptflow.azure import PFClient

# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

In [10]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

Populate the `config.json` file with the subscription_id, resource_group, and workspace_name.

In [ ]:
config_path = "../config.json"
pf_azure_client = PFClient.from_config(credential=credential, path=config_path)

Add the runtime from the AI Studio that will be used for the cloud batch runs.

In [ ]:
# Update the runtime to the name of the runtime you created previously
runtime = "automatic"
# load flow
flow = "../contoso-chat"
# load data
data = "../data/salestestdata.jsonl"

In [ ]:
# get current time stamp for run name
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y_%m_%d_%H%M%S")
run_name = timestamp+"chat_base_run"
print(run_name)

Create a base run to use as the variant for the evaluation runs. 

_NOTE: If you get "'An existing connection was forcibly closed by the remote host'" run the cell again._

In [ ]:
# create base run in Azure Ai Studio
base_run = pf_azure_client.run(
    flow=flow,
    data=data,
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
    },
    runtime=runtime,
    # create a display name as current datetime
    display_name=run_name,
    name=run_name
)
print(base_run)

In [ ]:
pf_azure_client.stream(base_run)

In [ ]:
details = pf_azure_client.get_details(base_run)
details.head(10)

# Cloud Eval run on Json Data

In [ ]:
eval_flow = "multi_flow/"
data = "../data/salestestdata.jsonl"
run_name = timestamp+"chat_eval_run"
print(run_name)

eval_run_variant = pf_azure_client.run(
    flow=eval_flow,
    data=data,  # path to the data file
    run=base_run,  # use run as the variant
    column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
        "context": "${run.outputs.context}",
        # reference the run's output
        "answer": "${run.outputs.answer}",
    },
    runtime=runtime,
    display_name=run_name,
    name=run_name
)

In [ ]:
pf_azure_client.stream(eval_run_variant)

In [ ]:
details = pf_azure_client.get_details(eval_run_variant)
details.head(10)

In [ ]:

metrics = pf_azure_client.get_metrics(eval_run_variant)
print(json.dumps(metrics, indent=4))

In [ ]:
pf_azure_client.visualize([base_run, eval_run_variant])